In [ ]:
import mne
import os
import scipy.io
import listen_italian_functions
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tnrange, tqdm_notebook

data_path = os.path.dirname(os.path.dirname(os.getcwd()))

subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
                'Silvia','Silvia2','Tommaso']
#subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
#                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale',
#                'Silvia','Silvia2','Tommaso']

In [ ]:
remove_first = 0.5 #seconds

# let's explore some frequency bands
iter_freqs = [
    ('Delta', 1, 3),
    ('Theta', 4, 7),
    ('Alpha', 8, 12),
    ('Beta', 13, 25),
    ('Gamma', 30, 40)
]

iter_freqs = [
    ('fr', 1, 3),
    #('fr', 2, 4),
    #('fr', 3, 5),
    ('fr', 4, 6)
    #('fr', 5, 7),
    #('fr', 6, 8),
    #('fr', 7, 9),
    #('fr', 8, 10),
    #('fr', 9, 11),
    #('fr', 10, 12)
]
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
condition = ['Hyper','Normal','Hypo', 'All']
condition_dummy = ['hyper','normal','hypo']

delay = np.arange(0,1.1,0.1)


indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))   

extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'Hyper': 1,'Normal': 2,'Hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
        
for s in range(0,len(subject_name)):
    frames = []
    for d in delay:        

        Ocoh = listen_italian_functions.coherence_preprocess_delay(GA_epoches[s],remove_first,d,trial_len,
                                                                        extra_channels,eeg_channles,
                                                                        info,ch_names,event_id,iter_freqs,indices,condition)

            
            
        for i in range(0,len(condition)):
            # mean or median of the surrogate distribution
            coh=Ocoh[:,:,i]
            for fr in range(0,len(iter_freqs)):
                a = str(iter_freqs[fr][0])+ ' '+str(iter_freqs[fr][1])+' - '+str(iter_freqs[fr][2])+'Hz'
                cc = np.split(coh[:,fr], len(features))
                for f in range(0,len(features)):
                    feature = features[f]
                    if condition[i] != 'All':
                        df = pd.DataFrame({'Condition':condition[i],'Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                       'Data':[cc[f].flatten()],'noTrials':GA_epoches[s][condition_dummy[i]].get_data().shape[0],})
                    else:
                        df = pd.DataFrame({'Condition':condition[i],'Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                       'Data':[cc[f].flatten()],'noTrials':GA_epoches[s].get_data().shape[0],})

            
                    
                    frames.append(df) 
    print(d)
                
    data=pd.concat((frames),axis=0)
    save_path = data_path + '/python/data/1-12Hz_frequency_bands_in2hzStep_coherence/coherence-trialLen-'\
    +str(trial_len)+'-removedFirst-'+str(remove_first)+'s-'+subject_name[s]
    data.to_pickle(save_path)

In [ ]:
remove_first = 0.5 #seconds
iter_freqs = [
    ('fr', 1, 3),
    ('fr', 4, 6)
]
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
delay = np.arange(0,1.1,0.1)



indices = (np.repeat([np.arange(59,len(features)+59)],59),np.tile(np.arange(0,59),len(features)))  
extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'hyper': 1,'normal': 2,'hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
    
for s in range(0,len(subject_name)):
    frames = []
    for d in delay:       
        surrogate_coh = listen_italian_functions.coherence_preprocess_delay_surrogate(GA_epoches[s],remove_first,
                                                                                      d,trial_len,extra_channels,
                                                                                      eeg_channles,info,ch_names,
                                                                                      event_id,iter_freqs,indices)
        
        # mean or median of the surrogate distribution
        coh=surrogate_coh.mean(axis=2)
        
        for fr in range(0,len(iter_freqs)):
            a = str(iter_freqs[fr][0])+ ' '+str(iter_freqs[fr][1])+' - '+str(iter_freqs[fr][2])+'Hz'
            cc = np.split(coh[:,fr], len(features))
            for f in range(0,len(features)):
                feature = features[f]
                df = pd.DataFrame({'Condition':'All','Freq':a,'Delay':d,'Subject':subject_name[s],'Feature':feature,
                                   'Surrogate':[cc[f].flatten()]})
                frames.append(df)                
    data=pd.concat((frames),axis=0)
    save_path = data_path + '/python/data/SurrogateCoherence/SurrogateCoherence-trialLen-'+str(trial_len)+'-removedFirst-'\
    +str(remove_first)+'s-'+subject_name[s]
    data.to_pickle(save_path)     
        
    

In [ ]:
# coherence investigate

In [ ]:
a=[]
for s in subject_name:
    a.append(pd.read_pickle(save_path+'-'+s))

data = pd.concat((a),axis=0)
data['Delay']=data['Delay'] - 0.5
data['Delay']=data['Delay'].round(decimals=1)

fr = data.Freq.unique()
print(fr)
data.set_index(['Feature','Freq','Condition','Delay','Subject'], inplace=True)
data.sort_index(inplace=True)

a  = data['Data'].apply(lambda x: np.mean(x))
data['Coherence'] = pd.Series(a, index=data.index)

data.head(10)

In [ ]:
condition = ['Hyper','Normal','Hypo','All']
a=pd.DataFrame(data.loc['envelop',fr[0],condition,0,subject_name])
a=a.groupby(['Subject','Condition']).sum()['noTrials'].unstack().plot(kind='bar',figsize=(20,5),grid=True)